In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import time
import json
import pandas as pd
import numpy as np
import os
import datetime
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods

In [51]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/test_dataset/Helpdesk_mini.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

# Extract the filename from the file path
filename = os.path.basename(file)

# Remove the file extension from the filename
dataName = os.path.splitext(filename)[0]

#dataName = 'Helpdesk_mini'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
print('data prepared done')



start_time = time.time()
print("Test Context Data")
#print(d.test_orig.contextdata)
basic_model = m.train(d.train,{"epochs": 0, "early_stop": 0})

Num events is 300
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 0
Test: 300
data prepared done
Test Context Data


In [52]:
connect_symbol="-"
if '/' in d.logfile.get_data()['completeTime'][0]:
    connect_symbol='/'
formats = [f"%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S%z", f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S',f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S.%f']

for timeformat in formats:
    try:
        pd.to_datetime(d.logfile.get_data()['completeTime'], format=timeformat,exact=True)
        print(d.logfile.get_data()['completeTime'])
        print("The time format is:", timeformat)
        break
    except ValueError:
        continue
print('timeformat',timeformat)

0      2010/01/13 07:40:25.000
1      2010/01/13 11:26:04.000
2      2010/01/13 11:30:37.000
3      2010/01/13 12:09:31.000
4      2010/01/13 16:25:25.000
                ...           
295    2010/03/09 07:06:03.000
296    2010/03/09 07:06:12.000
297    2010/03/09 07:18:19.000
298    2010/03/09 07:18:29.000
299    2010/03/09 08:08:53.000
Name: completeTime, Length: 300, dtype: object
The time format is: %Y/%m/%d %H:%M:%S
timeformat %Y/%m/%d %H:%M:%S


## make functions

In [ ]:
def add_data_to_test_orig(self,percentages, timeformat=None, ratio=None):
    print('original dataset size', self.logfile.get_data().shape[0])
    periods_logfile = self.logfile.split_percentages(percentages)
    new_rows = self.modify_sampled_rows_completetime(periods_logfile,timeformat,ratio)

    data_new = pd.concat([periods_logfile[0].get_data(), periods_logfile[1].get_data(), periods_logfile[2].get_data(),new_rows], ignore_index=True)
    # data_new = data_new.sort_values(by="completeTime", key=lambda x: pd.to_datetime(x, timeformat))
    data_new.sort_values(by="completeTime").reset_index()
    self.test_orig.contextdata=data_new
    print('modified dataset size', self.test_orig.get_data().shape[0])

In [7]:
periods_logfile = d.logfile.split_percentages([17.5,50])

0.175
Train: 53
Test: 247
period_logfile0 53
0.5
Train: 124
Test: 123
period_logfile1 124
period_logfile1 123


In [10]:
def modify_sampled_rows_completetime(self, periods_logfile, timeformat, ratio):
        
    sampled_rows = periods_logfile[0].get_data().sample(frac=ratio, random_state=42)
    
    df2 = periods_logfile[2].get_data()
    # Step 1: Get the earliest and latest timestamps from df2
    earliest_time = pd.to_datetime(df2['completeTime']).min()
    latest_time = pd.to_datetime(df2['completeTime']).max()

    # Step 2: Calculate the time range
    time_range = latest_time - earliest_time

    new_rows = sampled_rows.copy(deep=True)

    # Step 3: Generate random timestamps within the time range for each row in sampled_rows
    new_rows['completeTime'] = np.random.choice(pd.date_range(start=earliest_time, end=latest_time), size=len(sampled_rows))

    # Step 4: Update the values in the completeTime column of sampled_rows
    new_rows['completeTime'] = new_rows['completeTime'].dt.strftime(timeformat)

    # Step 1: Calculate the time difference
    time_difference = pd.to_datetime(new_rows['completeTime']) - pd.to_datetime(sampled_rows['completeTime'])
    # Step 2: Apply the time difference to the other columns
    for col in sampled_rows.columns:
        if col.startswith('completeTime_Prev'):
            zero_rows= sampled_rows[col]==0
            time_difference[zero_rows]= datetime.timedelta(days=0)
            new_rows[col] = ((pd.to_datetime(sampled_rows[col].replace(0,np.nan)) + time_difference).dt.strftime(timeformat)).fillna(0).apply(lambda x: int(x) if isinstance(x,float) else x)
            #new_rows[col] = pd.to_datetime(new_rows[col], format=timeformat)
    return new_rows

In [12]:
new_rows = modify_sampled_rows_completetime(d.logfile, periods_logfile, timeformat, 0.5)

In [13]:
new_rows

,index,event_Prev9,role_Prev9,completeTime_Prev9,case_Prev9,event_Prev8,role_Prev8,completeTime_Prev8,case_Prev8,event_Prev7,...,completeTime_Prev1,case_Prev1,event_Prev0,role_Prev0,completeTime_Prev0,case_Prev0,event,role,completeTime,case
19.0,145,0,0,0,43,0,0,0,43,0,...,2010/01/13 11:26:04,43,6,6,2010/01/19 08:26:05,43,5,6,2010/02/28 09:03:38,43
41.0,60,0,0,0,17,0,0,0,17,0,...,2010/01/13 12:09:31,17,6,6,2010/01/26 08:50:48,17,5,6,2010/03/08 09:03:38,17
47.0,235,0,0,0,70,0,0,0,70,0,...,0,70,1,6,2010/01/13 11:30:37,70,1,6,2010/03/08 09:03:38,70
12.0,223,0,0,0,67,0,0,0,67,0,...,2010/01/13 16:25:38,67,6,6,2010/01/14 10:42:31,67,5,6,2010/03/08 09:03:38,67
43.0,26,0,0,0,8,0,0,0,8,0,...,0,8,1,6,2010/01/26 10:36:21,8,6,6,2010/03/08 09:03:38,8
5.0,221,0,0,0,67,0,0,0,67,0,...,0,67,1,6,2010/01/13 16:25:25,67,1,6,2010/02/21 09:03:38,67
17.0,95,0,0,0,27,0,0,0,27,0,...,0,27,0,0,0,27,1,6,2010/02/21 09:03:38,27
50.0,28,0,0,0,8,0,0,0,8,0,...,2010/01/26 10:36:30,8,5,6,2010/01/26 15:06:29,8,6,6,2010/03/06 09:03:38,8
3.0,58,0,0,0,17,0,0,0,17,0,...,0,17,0,0,0,17,1,6,2010/02/19 09:03:38,17
32.0,80,0,0,0,23,0,0,0,23,0,...,0,23,0,0,0,23,1,6,2010/02/26 09:03:38,23


In [14]:
periods_logfile[2].get_data().shape[0]

123

In [ ]:
sampled_rows = periods_logfile[0].get_data().sample(frac=0.3, random_state=42)

In [42]:
import random
period_logfile_1 = periods_logfile[0].get_data()
print(len(period_logfile_1))
ratio=0.3
n=int(ratio*periods_logfile[2].get_data().shape[0]) # number of rows to sample
random_state = 42  # Fixed random state

if n >= len(period_logfile_1):
    # Replicate the dataset n // size times
    sampled_data = period_logfile_1.sample(n=len(period_logfile_1), replace=True, random_state=random_state).reset_index(drop=True)
    num_remaining = n - len(period_logfile_1)
    remaining_data = period_logfile_1.sample(n=num_remaining, replace=True, random_state=random_state).reset_index(drop=True)
    sampled_data = pd.concat([sampled_data] * (n // len(period_logfile_1)) + [remaining_data], ignore_index=True)
else:
    sampled_data = period_logfile_1.sample(n=n, replace=False, random_state=random_state).reset_index(drop=True)

print(sampled_data)


53
    index  event_Prev9  role_Prev9  completeTime_Prev9  case_Prev9  \
0     145            0           0                   0          43   
1      60            0           0                   0          17   
2     235            0           0                   0          70   
3     223            0           0                   0          67   
4      26            0           0                   0           8   
5     221            0           0                   0          67   
6      95            0           0                   0          27   
7      28            0           0                   0           8   
8      58            0           0                   0          17   
9      80            0           0                   0          23   
10     63            0           0                   0          18   
11    106            0           0                   0          31   
12      2            0           0                   0           2   
13    262        

In [23]:
period_logfile_2_size = periods_logfile[2].get_data().shape[0]

In [34]:
ratio = 0.99

In [38]:
sampled_rows_size = (ratio*period_logfile_2_size/(1-ratio))
sampled_rows_size/(period_logfile_2_size+sampled_rows_size)

0.99

In [47]:
d.add_data_to_test_orig([17.5,50],timeformat=timeformat, ratio=0.2)

original dataset size 300
0.175
Train: 53
Test: 247
period_logfile0 53
0.5
Train: 124
Test: 123
period_logfile1 124
period_logfile1 123
modified dataset size 300


## check add_data_to_test_orig

In [14]:
periods_logfile = d.logfile.split_percentages([20,50])

0.2
Train: 60
Test: 240
period_logfile0 60
0.5
Train: 120
Test: 120
period_logfile1 120


In [16]:
periods_logfile[1].get_data().shape[0]

120

In [18]:
def modify_sampled_rows_completetime(periods_logfile, timeformat, ratio):

    #sampled_rows = sampled_rows = periods_logfile[0].get_data().sample(frac=ratio, random_state=42)
    period_logfile_0 = periods_logfile[0].get_data()

    period_logfile_2_size = periods_logfile[2].get_data().shape[0]
    n = int(ratio*period_logfile_2_size)  # Number of rows to sample
    random_state = 42  # Fixed random state

    if n >= len(period_logfile_0):
        # Replicate the dataset n // size times
        sampled_rows = period_logfile_0.sample(n=len(period_logfile_0), replace=True, random_state=random_state).reset_index(drop=True)
        num_remaining = n - len(period_logfile_0)
        remaining_rows = period_logfile_0.sample(n=num_remaining, replace=True, random_state=random_state).reset_index(drop=True)
        sampled_rows = pd.concat([sampled_rows] * (n // len(period_logfile_0)) + [remaining_rows], ignore_index=True)
    else:
        sampled_rows = period_logfile_0.sample(n=n, replace=False, random_state=random_state).reset_index(drop=True)

    df2 = periods_logfile[2].get_data()
    # Step 1: Get the earliest and latest timestamps from df2
    earliest_time = pd.to_datetime(df2['completeTime']).min()
    latest_time = pd.to_datetime(df2['completeTime']).max()

    new_rows = sampled_rows.copy(deep=True)

    # Step 3: Generate random timestamps within the time range for each row in sampled_rows
    new_rows['completeTime'] = np.random.choice(pd.date_range(start=earliest_time, end=latest_time), size=len(sampled_rows))

    # Step 4: Update the values in the completeTime column of sampled_rows
    # Convert 'completeTime' to datetime data type
    new_rows['completeTime'] = pd.to_datetime(new_rows['completeTime'])
    new_rows['completeTime'] = new_rows['completeTime'].dt.strftime(timeformat)

    # Step 1: Calculate the time difference
    time_difference = pd.to_datetime(new_rows['completeTime']) - pd.to_datetime(sampled_rows['completeTime'])
    # Step 2: Apply the time difference to the other columns
    for col in sampled_rows.columns:
        if col.startswith('completeTime_Prev'):
            zero_rows= sampled_rows[col]==0
            time_difference[zero_rows]= datetime.timedelta(days=0)
            new_rows[col] = ((pd.to_datetime(sampled_rows[col].replace(0,np.nan)) + time_difference).dt.strftime(timeformat)).fillna(0).apply(lambda x: int(x) if isinstance(x,float) else x)
            #new_rows[col] = pd.to_datetime(new_rows[col], format=timeformat)
    return new_rows

In [19]:
new_rows = modify_sampled_rows_completetime(periods_logfile, timeformat, 0.5)

In [21]:
new_rows.shape[0]

60

In [23]:
random_state = 42
period_logfile_2 = periods_logfile[2].get_data()
remained_rows_size = periods_logfile[2].get_data().shape[0]-new_rows.shape[0]
print('remained_rows_size',remained_rows_size)
if remained_rows_size != 0:
    remained_rows = period_logfile_2.sample(n=periods_logfile[2].get_data().shape[0]-new_rows.shape[0], replace=True, random_state=random_state).reset_index(drop=True)
    #print('n',n)
    data_new = pd.concat([periods_logfile[0].get_data(), periods_logfile[1].get_data(), remained_rows,new_rows], ignore_index=True)
else:
    data_new =  pd.concat([periods_logfile[0].get_data(), periods_logfile[1].get_data(), new_rows], ignore_index=True)
# data_new = data_new.sort_values(by="completeTime", key=lambda x: pd.to_datetime(x, timeformat))
data_new.sort_values(by="completeTime").reset_index()
d.logfile.contextdata=data_new
print('modified dataset size', d.test_orig.get_data().shape[0])

remained_rows_size 60
modified dataset size 300


In [ ]:
def add_data_to_test_orig(d,percentages, timeformat=None, ratio=None):
    print('original dataset size', d.logfile.get_data().shape[0])
    periods_logfile = d.logfile.split_percentages(percentages)
    new_rows = d.modify_sampled_rows_completetime(periods_logfile,timeformat,ratio)
    random_state = 42
    period_logfile_2 = periods_logfile[2].get_data()
    remained_rows_size = periods_logfile[2].get_data().shape[0]-new_rows.shape[0]
    if remained_rows_size != 0:
        remained_rows = period_logfile_2.sample(n=periods_logfile[2].get_data().shape[0]-new_rows.shape[0], replace=True, random_state=random_state).reset_index(drop=True)
        data_new = pd.concat([periods_logfile[0].get_data(), periods_logfile[1].get_data(), remained_rows,new_rows], ignore_index=True)
    else:
        data_new =  pd.concat([periods_logfile[0].get_data(), periods_logfile[1].get_data(), new_rows], ignore_index=True)
    # data_new = data_new.sort_values(by="completeTime", key=lambda x: pd.to_datetime(x, timeformat))
    data_new.sort_values(by="completeTime").reset_index()
    d.test_orig.contextdata=data_new
    print('modified dataset size', d.test_orig.get_data().shape[0])

## whether data is added?

In [53]:
df = d.logfile.get_data()

In [54]:
d.add_data_to_test_orig([40,50],timeformat,0.4)

original dataset size 300
0.4
Train: 120
Test: 180
period_logfile0 120
0.5
Train: 90
Test: 90
period_logfile1 90
sampled_rows 36 new_rows 36
new_rows_size 36 period_logfile_2_size 90
remained_rows_size 54 periods_logfile[2].get_data().shape[0]-new_rows.shape[0] 54
modified dataset size 300


In [55]:
modified_df = d.logfile.get_data()

In [56]:
columns_to_drop = [col for col in df.columns if col.startswith('completeTime')]
# Add 'index' column to the list of columns to drop
columns_to_drop.append('index')
df = df.drop(columns=columns_to_drop)
modified_df = modified_df.drop(columns=columns_to_drop)

### how many rows in period_logfile 2 are from period_logfile_0

In [57]:
count = 0
for row in modified_df.iloc[210:].itertuples(index=False, name=None):
    for row2 in df.iloc[:120].itertuples(index=False, name=None):
        if row == row2:
            #print(row)
            count+=1
            break
print(count)

36


### how many rows in period_logfile 2 are from period_logfile_2

In [58]:
count = 0
for row in modified_df.iloc[210:].itertuples(index=False, name=None):
    for row2 in df.iloc[210:].itertuples(index=False, name=None):
        if row == row2:
            #print(row)
            count+=1
            break
print(count)

54


In [91]:
df.iloc[0,:].values

array([ 0,  0, 59,  0,  0, 59,  0,  0, 59,  0,  0, 59,  0,  0, 59,  0,  0,
       59,  0,  0, 59,  0,  0, 59,  0,  0, 59,  0,  0, 59,  1,  6, 59])

In [77]:
# Merge dataframes with indicator
merged = pd.merge(df.reset_index(), modified_df.reset_index(), on=df.columns.tolist(), how='inner', indicator=True)

# Filter rows where _merge is 'both'
common_rows = merged[merged['_merge'] == 'both']

# Print row numbers of common rows in each dataframe
print("Common rows in 'df':", common_rows['index_x'].tolist())
print("Common rows in 'modified_df':", common_rows['index_y'].tolist())

Common rows in 'df': [0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 10, 11, 12, 13, 13, 14, 14, 14, 15, 16, 17, 17, 18, 19, 20, 20, 20, 20, 21, 21, 21, 22, 23, 23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 32, 32, 33, 34, 34, 35, 35, 36, 37, 37, 38, 38, 39, 40, 41, 41, 42, 43, 43, 44, 45, 46, 46, 46, 47, 48, 48, 49, 49, 50, 50, 50, 51, 51, 52, 52, 52, 53, 53, 54, 54, 55, 56, 57, 57, 58, 58, 59, 59, 59, 59, 59, 60, 60, 61, 61, 61, 61, 62, 63, 63, 63, 64, 65, 66, 67, 68, 69, 70, 70, 71, 71, 72, 72, 73, 74, 74, 74, 75, 75, 76, 77, 77, 78, 79, 79, 80, 80, 81, 82, 82, 83, 83, 84, 85, 86, 86, 87, 87, 87, 88, 88, 88, 89, 89, 90, 90, 91, 91, 91, 92, 92, 93, 94, 95, 96, 97, 98, 99, 99, 100, 100, 101, 102, 102, 102, 103, 103, 103, 104, 105, 105, 106, 106, 107, 107, 107, 107, 108, 109, 110, 110, 111, 112, 113, 113, 114, 115, 115, 116, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 14

## check whether pl2 exist in pl0

In [143]:
filtered_rows = common_rows[(common_rows['index_x'] > 120) & (common_rows['index_x'] < 210)]

print("Filtered common rows in 'df':", filtered_rows['index_x'].tolist())
print("Corresponding rows in 'modified_df':", filtered_rows['index_y'].tolist())

Filtered common rows in 'df': [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
Corresponding rows in 'modified_df': [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]


In [142]:
np.sum(common_rows['index_x']>=210)

0

In [67]:
common_rows = pd.merge(df.iloc[200:], modified_df.iloc[200:], how='inner')

if common_rows.empty:
    print("There are no common rows between the two DataFrames.")
else:
    print("The two DataFrames have common rows.")
    print(common_rows)


The two DataFrames have common rows.
   event_Prev9  role_Prev9  case_Prev9  event_Prev8  role_Prev8  case_Prev8  \
0            0           0          79            0           0          79   
1            0           0          79            0           0          79   
2            0           0           4            0           0           4   
3            0           0          10            0           0          10   
4            0           0          10            0           0          10   
5            0           0          52            0           0          52   
6            0           0          52            0           0          52   
7            0           0          37            0           0          37   
8            0           0           9            0           0           9   
9            0           0           9            0           0           9   

   event_Prev7  role_Prev7  case_Prev7  event_Prev6  ...  case_Prev2  \
0            0       

In [31]:
are_identical = df.equals(modified_df)
print(are_identical) 

False
